# 20_adam_crab_comparison â€” CRAB + Adam comparison


Optimization study following the GRAPE_CRAB_ADAM guide. Time is tracked in microseconds and control amplitudes in rad/Âµs. Each mode saves reproducible artifacts (config, pulses, metadata) for downstream figure generation.


## Objective & Runtime Choices
- Terminal mode: weighed direct terminal fidelity versus time-distributed tracking; kept the final-time fidelity objective to mirror the guide and baseline notebooks.
- Ensemble mode: compared Monte Carlo sampling with replacement to a deterministic 51Ã—51 tensor grid over Î² and Doppler; selected the full grid for reproducible statistics.
- Adiabatic mode: evaluated finite-difference adiabatic penalties versus instantaneous eigen-projector tracking; implemented the projector-based integral from the guide to follow the eigen-path explicitly.
- Runtime tracking: contrasted 	ime.perf_counter and 	ime.process_time; chose perf_counter to capture wall-clock iteration durations including physics kernel calls.


In [57]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / "src").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
os.chdir(PROJECT_ROOT)



In [58]:
import json
import math
import sys
import time
from pathlib import Path
from typing import Dict, Tuple

import numpy as np

from src.qoc_common_crab import (
    load_baseline_crab,
    controls_from_coeffs,
)
from src.qoc_common import (
    terminal_cost_and_grad,
    forward_rhos,
    U_step,
)
from src.crab_notebook_utils import (
    ensure_dir,
    json_ready,
    collect_versions,
    validate_time_grid,
    assert_finite,
    penalty_terms,
    ground_state_projectors,
    SIGMA_X,
    SIGMA_Z,
)

np.set_printoptions(suppress=True, linewidth=120)
NOTEBOOK_NAME = "20_adam_crab_comparison.ipynb"


In [59]:
arrays, policy = load_baseline_crab()

t = arrays["t"].astype(float)
dt = float(arrays["dt"])
T = float(arrays["T"])
Nt = int(arrays["Nt"])

validate_time_grid(t, dt, T)

Omega0 = arrays["Omega0"].astype(float)
Delta0 = arrays["Delta0"].astype(float)
basis_omega = arrays["CRAB_BASIS_OMEGA"].astype(float)
basis_delta = arrays["CRAB_BASIS_DELTA"].astype(float)
rho0 = arrays["rho0"]
target = arrays["target"]
seed = int(arrays["SEED"])

assert Omega0.shape == (Nt,)
assert Delta0.shape == (Nt,)
assert basis_omega.shape[0] == Nt
assert basis_delta.shape[0] == Nt

assert_finite("Omega0", Omega0)
assert_finite("Delta0", Delta0)

penalty_config = policy.get("penalties", {})
power_weight_default = float(penalty_config.get("power_weight", 0.0))
neg_weight_default = float(penalty_config.get("neg_weight", 0.0))
neg_kappa_default = float(penalty_config.get("neg_kappa", 10.0))

delta_max = float(np.max(np.abs(Delta0)))
print(f"Loaded baseline: Nt={Nt}, dt={dt:.6f} us, T={T:.6f} us, max|Î”0|={delta_max:.3f} rad/us")


Loaded baseline: Nt=201, dt=0.000500 us, T=0.100000 us, max|Î”0|=183.260 rad/us


In [60]:
RUN_NAME = "20_adam_crab_comparison"
RESULTS_DIR = ensure_dir(Path("results") / RUN_NAME)
beta_grid = np.linspace(0.9, 1.1, 5)
doppler_grid = np.linspace(-delta_max, delta_max, 5)

CONFIG = {
    "notebook": NOTEBOOK_NAME,
    "run_name": RUN_NAME,
    "seed": seed,
    "units": {"time": "us", "amplitude": "rad/us"},
    "time_grid": {"Nt": Nt, "dt_us": dt, "T_us": T},
    "paths": {
        "results": str(RESULTS_DIR),
        "figures": "figures/21_optimization_figures",
        "baseline_arrays": "data/baselines/_baseline_crab/arrays.npz",
        "baseline_metadata": "data/baselines/_baseline_crab/metadata.json",
    },
    "penalties": {
        "power_weight": power_weight_default,
        "neg_weight": neg_weight_default,
        "neg_kappa": neg_kappa_default,
    },
    "adam": {
        "lr": 8e-2,
        "beta1": 0.9,
        "beta2": 0.999,
        "eps": 1e-8,
    },
    "stopping": {
        "max_iters": 1000,
        "target_cost": 1e-5,
        "max_oracle_calls": 400000,
    },
    "ensemble": {
        "beta_grid": beta_grid.tolist(),
        "doppler_grid": doppler_grid.tolist(),
        "weights": "uniform",
    },
    "modes": {
        "terminal": {"objective": "terminal_fidelity"},
        "ensemble": {"objective": "ensemble_expected_fidelity"},
        "adiabatic": {"objective": "adiabatic_path_tracking"},
    },
}

print(json.dumps({k: CONFIG[k] for k in ("penalties", "adam", "stopping")}, indent=2))


{
  "penalties": {
    "power_weight": 0.0001,
    "neg_weight": 1.0,
    "neg_kappa": 1.0
  },
  "adam": {
    "lr": 0.08,
    "beta1": 0.9,
    "beta2": 0.999,
    "eps": 1e-08
  },
  "stopping": {
    "max_iters": 1000,
    "target_cost": 1e-05,
    "max_oracle_calls": 400000
  }
}


In [61]:
SIGMA_X_HALF = 0.5 * SIGMA_X
SIGMA_Z_HALF = 0.5 * SIGMA_Z
penalty_power_weight = float(CONFIG["penalties"]["power_weight"])
penalty_neg_weight = float(CONFIG["penalties"]["neg_weight"])
penalty_neg_kappa = float(CONFIG["penalties"]["neg_kappa"])
beta_vals = np.asarray(CONFIG["ensemble"]["beta_grid"], dtype=float)
doppler_vals = np.asarray(CONFIG["ensemble"]["doppler_grid"], dtype=float)
ensemble_size = beta_vals.size * doppler_vals.size
adam_lr = float(CONFIG["adam"]["lr"])
adam_beta1 = float(CONFIG["adam"]["beta1"])
adam_beta2 = float(CONFIG["adam"]["beta2"])
adam_eps = float(CONFIG["adam"]["eps"])
stopping_cfg = CONFIG["stopping"]
def assemble_controls(coeffs_Omega: np.ndarray, coeffs_Delta: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    Omega, Delta = controls_from_coeffs(coeffs_Omega, coeffs_Delta, Omega0, Delta0, basis_omega, basis_delta)
    assert_finite("Omega", Omega)
    assert_finite("Delta", Delta)
    return Omega.astype(float, copy=False), Delta.astype(float, copy=False)
def evaluate_terminal(coeffs_Omega: np.ndarray, coeffs_Delta: np.ndarray) -> Dict[str, object]:
    Omega, Delta = assemble_controls(coeffs_Omega, coeffs_Delta)
    terminal_infidelity, gO_time, gD_time = terminal_cost_and_grad(
        Omega,
        Delta,
        rho0,
        dt,
        target,
        power_weight=0.0,
        neg_weight=0.0,
    )
    pen_power, pen_neg, grad_penalty_time = penalty_terms(
        Omega,
        dt,
        power_weight=penalty_power_weight,
        neg_weight=penalty_neg_weight,
        neg_kappa=penalty_neg_kappa,
    )
    gO_time_total = gO_time + grad_penalty_time
    gD_time_total = gD_time
    gO_coeff = basis_omega.T @ gO_time_total
    gD_coeff = basis_delta.T @ gD_time_total
    assert_finite("gO_coeff", gO_coeff)
    assert_finite("gD_coeff", gD_coeff)
    terminal_infidelity = float(terminal_infidelity)
    total_cost = terminal_infidelity + pen_power + pen_neg
    final_fidelity = float(np.clip(1.0 - terminal_infidelity, 0.0, 1.0))
    final_infidelity = 1.0 - final_fidelity
    return {
        "cost": float(total_cost),
        "final_infidelity": float(final_infidelity),
        "final_fidelity": float(final_fidelity),
        "penalties": {"power": float(pen_power), "neg": float(pen_neg)},
        "grad_coeffs": (gO_coeff, gD_coeff),
        "grad_time": (gO_time_total, gD_time_total),
        "controls": (Omega, Delta),
        "oracle_calls": 1,
        "extras": {},
    }
def evaluate_ensemble(coeffs_Omega: np.ndarray, coeffs_Delta: np.ndarray) -> Dict[str, object]:
    Omega, Delta = assemble_controls(coeffs_Omega, coeffs_Delta)
    base_infidelity, _, _ = terminal_cost_and_grad(
        Omega,
        Delta,
        rho0,
        dt,
        target,
        power_weight=0.0,
        neg_weight=0.0,
    )
    base_infidelity = float(base_infidelity)
    base_fidelity = float(np.clip(1.0 - base_infidelity, 0.0, 1.0))
    base_infidelity = 1.0 - base_fidelity
    pen_power, pen_neg, grad_penalty_time = penalty_terms(
        Omega,
        dt,
        power_weight=penalty_power_weight,
        neg_weight=penalty_neg_weight,
        neg_kappa=penalty_neg_kappa,
    )
    gO_acc = np.zeros_like(Omega)
    gD_acc = np.zeros_like(Delta)
    inf_sum = 0.0
    fid_sum = 0.0
    fid_sq_sum = 0.0
    for beta in beta_vals:
        for shift in doppler_vals:
            Omega_mod = beta * Omega
            Delta_mod = Delta + shift
            terminal_infidelity, gO_time, gD_time = terminal_cost_and_grad(
                Omega_mod,
                Delta_mod,
                rho0,
                dt,
                target,
                power_weight=0.0,
                neg_weight=0.0,
            )
            gO_acc += beta * gO_time
            gD_acc += gD_time
            raw_infidelity = float(terminal_infidelity)
            inf_sum += raw_infidelity
            sample_fidelity = float(np.clip(1.0 - raw_infidelity, 0.0, 1.0))
            fid_sum += sample_fidelity
            fid_sq_sum += sample_fidelity * sample_fidelity
    inv_members = 1.0 / float(ensemble_size)
    mean_final_infidelity = float(np.clip(inf_sum * inv_members, 0.0, 1.0))
    mean_final_fidelity = float(np.clip(fid_sum * inv_members, 0.0, 1.0))
    variance = max(fid_sq_sum * inv_members - mean_final_fidelity * mean_final_fidelity, 0.0)
    std_final_fidelity = math.sqrt(variance)
    gO_time_mean = gO_acc * inv_members + grad_penalty_time
    gD_time_mean = gD_acc * inv_members
    gO_coeff = basis_omega.T @ gO_time_mean
    gD_coeff = basis_delta.T @ gD_time_mean
    assert_finite("gO_coeff ensemble", gO_coeff)
    assert_finite("gD_coeff ensemble", gD_coeff)
    total_cost = mean_final_infidelity + pen_power + pen_neg
    return {
        "cost": float(total_cost),
        "final_infidelity": float(base_infidelity),
        "final_fidelity": float(base_fidelity),
        "mean_final_infidelity": float(mean_final_infidelity),
        "mean_final_fidelity": float(mean_final_fidelity),
        "penalties": {"power": float(pen_power), "neg": float(pen_neg)},
        "grad_coeffs": (gO_coeff, gD_coeff),
        "grad_time": (gO_time_mean, gD_time_mean),
        "controls": (Omega, Delta),
        "oracle_calls": ensemble_size,
        "extras": {
            "mean_final_fidelity": float(mean_final_fidelity),
            "std_final_fidelity": float(std_final_fidelity),
            "mean_final_infidelity": float(mean_final_infidelity),
            "final_fidelity": float(base_fidelity),
            "final_infidelity": float(base_infidelity),
        },
    }
def evaluate_adiabatic(coeffs_Omega: np.ndarray, coeffs_Delta: np.ndarray) -> Dict[str, object]:
    Omega, Delta = assemble_controls(coeffs_Omega, coeffs_Delta)
    pen_power, pen_neg, grad_penalty_time = penalty_terms(
        Omega,
        dt,
        power_weight=penalty_power_weight,
        neg_weight=penalty_neg_weight,
        neg_kappa=penalty_neg_kappa,
    )
    rhos = forward_rhos(Omega, Delta, rho0, dt)
    projectors = ground_state_projectors(Omega, Delta)
    overlaps = np.real(np.einsum('kij,kji->k', projectors, rhos))
    path_fidelity = float(np.clip((dt / T) * overlaps.sum(), 0.0, 1.0))
    path_infidelity = 1.0 - path_fidelity
    final_state = rhos[-1]
    final_fidelity = float(np.clip(np.real(np.trace(final_state @ target)), 0.0, 1.0))
    final_infidelity = 1.0 - final_fidelity
    Nt_local = Omega.size
    lams = np.zeros((Nt_local, 2, 2), dtype=np.complex128)
    propagators = [U_step(float(Omega[k]), float(Delta[k]), dt) for k in range(Nt_local - 1)]
    for k in range(Nt_local - 2, -1, -1):
        lam_next = lams[k + 1]
        lams[k] = propagators[k].conj().T @ (lam_next + (dt / T) * projectors[k]) @ propagators[k]
    gO_time = np.zeros_like(Omega)
    gD_time = np.zeros_like(Delta)
    for k in range(Nt_local - 1):
        rho_k = rhos[k]
        lam_next = lams[k + 1]
        gO_time[k] = -np.imag(np.trace(lam_next @ (SIGMA_X_HALF @ rho_k - rho_k @ SIGMA_X_HALF))) * dt
        gD_time[k] = -np.imag(np.trace(lam_next @ (SIGMA_Z_HALF @ rho_k - rho_k @ SIGMA_Z_HALF))) * dt
    gO_time_total = gO_time + grad_penalty_time
    gD_time_total = gD_time
    gO_coeff = basis_omega.T @ gO_time_total
    gD_coeff = basis_delta.T @ gD_time_total
    assert_finite("gO_coeff adiabatic", gO_coeff)
    assert_finite("gD_coeff adiabatic", gD_coeff)
    total_cost = path_infidelity + pen_power + pen_neg
    return {
        "cost": float(total_cost),
        "final_infidelity": float(final_infidelity),
        "final_fidelity": float(final_fidelity),
        "path_infidelity": float(path_infidelity),
        "path_fidelity": float(path_fidelity),
        "penalties": {"power": float(pen_power), "neg": float(pen_neg)},
        "grad_coeffs": (gO_coeff, gD_coeff),
        "grad_time": (gO_time_total, gD_time_total),
        "controls": (Omega, Delta),
        "oracle_calls": 1,
        "extras": {
            "path_fidelity": float(path_fidelity),
            "path_infidelity": float(path_infidelity),
        },
    }
def init_history(mode_name: str) -> Dict[str, list]:
    history = {
        "iteration": [],
        "cost": [],
        "power_penalty": [],
        "neg_penalty": [],
        "grad_norm": [],
        "step_norm": [],
        "runtime_s": [],
        "oracle_calls": [],
    }
    if mode_name == "terminal":
        history["final_infidelity"] = []
        history["final_fidelity"] = []
    elif mode_name == "ensemble":
        history["mean_final_infidelity"] = []
        history["mean_final_fidelity"] = []
        history["std_final_fidelity"] = []
    elif mode_name == "adiabatic":
        history["final_infidelity"] = []
        history["final_fidelity"] = []
        history["path_fidelity"] = []
    return history
def run_mode(mode_name: str, evaluator) -> Tuple[Dict[str, list], Dict[str, object]]:
    max_iters = int(stopping_cfg["max_iters"])
    target_cost = float(stopping_cfg["target_cost"])
    max_calls = stopping_cfg.get("max_oracle_calls")
    coeffs_Omega = np.zeros(basis_omega.shape[1], dtype=float)
    coeffs_Delta = np.zeros(basis_delta.shape[1], dtype=float)
    mO = np.zeros_like(coeffs_Omega)
    vO = np.zeros_like(coeffs_Omega)
    mD = np.zeros_like(coeffs_Delta)
    vD = np.zeros_like(coeffs_Delta)
    history = init_history(mode_name)
    total_calls = 0
    final_eval = None
    start_time = time.perf_counter()
    for iteration in range(1, max_iters + 1):
        iter_start = time.perf_counter()
        eval_res = evaluator(coeffs_Omega, coeffs_Delta)
        total_calls += int(eval_res["oracle_calls"])
        gO_coeff, gD_coeff = eval_res["grad_coeffs"]
        grad_vec = np.concatenate([gO_coeff, gD_coeff])
        grad_norm = float(np.linalg.norm(grad_vec))
        done = eval_res["cost"] <= target_cost or (max_calls is not None and total_calls >= max_calls)
        step_Omega = np.zeros_like(coeffs_Omega)
        step_Delta = np.zeros_like(coeffs_Delta)
        if not done:
            mO[:] = adam_beta1 * mO + (1 - adam_beta1) * gO_coeff
            vO[:] = adam_beta2 * vO + (1 - adam_beta2) * (gO_coeff * gO_coeff)
            mD[:] = adam_beta1 * mD + (1 - adam_beta1) * gD_coeff
            vD[:] = adam_beta2 * vD + (1 - adam_beta2) * (gD_coeff * gD_coeff)
            mO_hat = mO / (1 - adam_beta1 ** iteration)
            vO_hat = vO / (1 - adam_beta2 ** iteration)
            mD_hat = mD / (1 - adam_beta1 ** iteration)
            vD_hat = vD / (1 - adam_beta2 ** iteration)
            step_Omega = -adam_lr * mO_hat / (np.sqrt(vO_hat) + adam_eps)
            step_Delta = -adam_lr * mD_hat / (np.sqrt(vD_hat) + adam_eps)
            coeffs_Omega = coeffs_Omega + step_Omega
            coeffs_Delta = coeffs_Delta + step_Delta
        step_norm = float(np.linalg.norm(np.concatenate([step_Omega, step_Delta])))
        iter_time = time.perf_counter() - iter_start
        history["iteration"].append(iteration)
        history["cost"].append(eval_res["cost"])
        history["power_penalty"].append(eval_res["penalties"]["power"])
        history["neg_penalty"].append(eval_res["penalties"]["neg"])
        history["grad_norm"].append(grad_norm)
        history["step_norm"].append(step_norm)
        history["runtime_s"].append(iter_time)
        history["oracle_calls"].append(total_calls)
        if mode_name == "terminal":
            history["final_infidelity"].append(eval_res["final_infidelity"])
            history["final_fidelity"].append(eval_res["final_fidelity"])
        elif mode_name == "ensemble":
            history["mean_final_infidelity"].append(eval_res["mean_final_infidelity"])
            history["mean_final_fidelity"].append(eval_res["mean_final_fidelity"])
            history["std_final_fidelity"].append(eval_res["extras"]["std_final_fidelity"])
        elif mode_name == "adiabatic":
            history["final_infidelity"].append(eval_res["final_infidelity"])
            history["final_fidelity"].append(eval_res["final_fidelity"])
            history["path_fidelity"].append(eval_res["path_fidelity"])
        final_eval = eval_res
        if done:
            break
    else:
        final_eval = evaluator(coeffs_Omega, coeffs_Delta)
        total_calls += int(final_eval["oracle_calls"])
    total_runtime = time.perf_counter() - start_time
    final_eval = final_eval.copy()
    final_eval["coeffs"] = (coeffs_Omega.astype(float, copy=True), coeffs_Delta.astype(float, copy=True))
    final_controls = tuple(arr.astype(float, copy=True) for arr in final_eval["controls"])
    final_eval["controls"] = final_controls
    final_eval["iterations"] = len(history["iteration"])
    final_eval["runtime_s"] = float(total_runtime)
    final_eval["oracle_calls_total"] = int(total_calls)
    return history, final_eval
def history_to_arrays(history: Dict[str, list]) -> Dict[str, np.ndarray]:
    return {key: np.asarray(values) for key, values in history.items()}


In [62]:
mode_histories: Dict[str, Dict[str, list]] = {}
mode_results: Dict[str, Dict[str, object]] = {}
for mode_name, evaluator in (
    ('terminal', evaluate_terminal),
    ('ensemble', evaluate_ensemble),
    ('adiabatic', evaluate_adiabatic),
):
    history, result = run_mode(mode_name, evaluator)
    mode_histories[mode_name] = history
    mode_results[mode_name] = result
    if mode_name == 'terminal':
        fid_summary = f"final_fidelity={result['final_fidelity']:.6f}"
    elif mode_name == 'ensemble':
        fid_summary = (
            f"final_fidelity={result['final_fidelity']:.6f} "
            f"mean_final_fidelity={result['mean_final_fidelity']:.6f} "
            f"(std={result['extras']['std_final_fidelity']:.6f})"
        )
    else:  # adiabatic
        fid_summary = (
            f"final_fidelity={result['final_fidelity']:.6f} "
            f"path_fidelity={result['path_fidelity']:.6f}"
        )
    print(
        f"[{mode_name}] iters={result['iterations']} cost={result['cost']:.6e} "
        f"{fid_summary} runtime={result['runtime_s']:.2f}s calls={result['oracle_calls_total']}"
    )


[terminal] iters=1000 cost=2.494450e-02 final_fidelity=0.999833 runtime=8.55s calls=1001
[ensemble] iters=1000 cost=8.074746e-02 final_fidelity=0.997871 mean_final_fidelity=0.990991 (std=0.011402) runtime=254.32s calls=25025
[adiabatic] iters=1000 cost=4.418770e-02 final_fidelity=0.995800 path_fidelity=0.983920 runtime=9.92s calls=1001


In [63]:
zero_coeffs_Omega = np.zeros(basis_omega.shape[1], dtype=float)
zero_coeffs_Delta = np.zeros(basis_delta.shape[1], dtype=float)
baseline_terminal = evaluate_terminal(zero_coeffs_Omega, zero_coeffs_Delta)
baseline_ensemble = evaluate_ensemble(zero_coeffs_Omega, zero_coeffs_Delta)
baseline_adiabatic = evaluate_adiabatic(zero_coeffs_Omega, zero_coeffs_Delta)
print('Baseline metrics (original pulses):')
print(
    f"  Terminal final fidelity: {baseline_terminal['final_fidelity']:.6f} "
    f"  Ensemble mean final fidelity: {baseline_ensemble['mean_final_fidelity']:.6f} "
    f" +/- {baseline_ensemble['extras']['std_final_fidelity']:.6f}"
    f"  path fidelity={baseline_adiabatic['path_fidelity']:.6f}"
)


Baseline metrics (original pulses):
  Terminal final fidelity: 0.934610   Ensemble mean final fidelity: 0.637018  +/- 0.252344  path fidelity=0.957364


In [64]:
config_to_save = json_ready({**CONFIG, "generated_at": time.strftime("%Y-%m-%d %H:%M:%S")})
(RESULTS_DIR / "config.json").write_text(json.dumps(config_to_save, indent=2), encoding="utf-8")
versions = collect_versions({"numpy": np})
versions["python"] = sys.version
(RESULTS_DIR / "library_versions.json").write_text(json.dumps(json_ready(versions), indent=2), encoding="utf-8")
pulses_payload = {
    "t": t,
    "omega_original": Omega0,
    "delta_original": Delta0,
}
for mode_name, result in mode_results.items():
    pulses_payload[f"omega_{mode_name}"] = result["controls"][0]
    pulses_payload[f"delta_{mode_name}"] = result["controls"][1]
np.savez_compressed(RESULTS_DIR / "pulses.npz", **pulses_payload)
original_metadata = {
    "notebook": NOTEBOOK_NAME,
    "description": "Baseline CRAB pulses before optimization",
    "terminal_final_fidelity": baseline_terminal["final_fidelity"],
    "terminal_final_infidelity": baseline_terminal["final_infidelity"],
    "terminal_cost": baseline_terminal["cost"],
    "ensemble_mean_final_fidelity": baseline_ensemble["mean_final_fidelity"],
    "ensemble_mean_final_infidelity": baseline_ensemble["mean_final_infidelity"],
    "ensemble_std_final_fidelity": baseline_ensemble["extras"]["std_final_fidelity"],
    "adiabatic_final_fidelity": baseline_adiabatic["final_fidelity"],
    "adiabatic_path_fidelity": baseline_adiabatic["path_fidelity"],
    "penalties": baseline_terminal["penalties"],
}
(RESULTS_DIR / "original_metadata.json").write_text(json.dumps(json_ready(original_metadata), indent=2), encoding="utf-8")
for mode_name, history in mode_histories.items():
    hist_arrays = history_to_arrays(history)
    np.savez_compressed(RESULTS_DIR / f"mode_{mode_name}_history.npz", **hist_arrays)
    result = mode_results[mode_name]
    np.savez_compressed(
        RESULTS_DIR / f"mode_{mode_name}_results.npz",
        omega=result["controls"][0],
        delta=result["controls"][1],
        coeffs_omega=result["coeffs"][0],
        coeffs_delta=result["coeffs"][1],
    )
    meta = {
        "mode": mode_name,
        "objective": CONFIG["modes"][mode_name]["objective"],
        "notebook": NOTEBOOK_NAME,
        "seed": seed,
        "iterations": result["iterations"],
        "runtime_s": result["runtime_s"],
        "oracle_calls": result["oracle_calls_total"],
        "final_cost": result["cost"],
        "penalties": result["penalties"],
        "adam": CONFIG["adam"],
        "stopping": CONFIG["stopping"],
    }
    if mode_name == "terminal":
        meta["final_fidelity"] = result["final_fidelity"]
        meta["final_infidelity"] = result["final_infidelity"]
    elif mode_name == "ensemble":
        meta["mean_final_fidelity"] = result["mean_final_fidelity"]
        meta["mean_final_infidelity"] = result["mean_final_infidelity"]
        meta["std_final_fidelity"] = result["extras"]["std_final_fidelity"]
        meta["ensemble_grid"] = {
            "beta_min": float(beta_vals.min()),
            "beta_max": float(beta_vals.max()),
            "doppler_min": float(doppler_vals.min()),
            "doppler_max": float(doppler_vals.max()),
            "grid_size": int(ensemble_size),
        }
    elif mode_name == "adiabatic":
        meta["final_fidelity"] = result["final_fidelity"]
        meta["final_infidelity"] = result["final_infidelity"]
        meta["path_fidelity"] = result["path_fidelity"]
        meta["path_infidelity"] = result["path_infidelity"]
    (RESULTS_DIR / f"mode_{mode_name}_metadata.json").write_text(json.dumps(json_ready(meta), indent=2), encoding="utf-8")
print("Artifacts in", RESULTS_DIR)
for path in sorted(RESULTS_DIR.iterdir()):
    print("  ", path.name)


Artifacts in results\20_adam_crab_comparison
   config.json
   library_versions.json
   mode_adiabatic_history.npz
   mode_adiabatic_metadata.json
   mode_adiabatic_results.npz
   mode_ensemble_history.npz
   mode_ensemble_metadata.json
   mode_ensemble_results.npz
   mode_terminal_history.npz
   mode_terminal_metadata.json
   mode_terminal_results.npz
   original_metadata.json
   pulses.npz


In [65]:
print("Final summary (costs and fidelities):")
for mode_name, result in mode_results.items():
    if mode_name == 'ensemble':
        fid_text = (
            f"final_fidelity={result['final_fidelity']:.6f} "
            f"mean_final_fidelity={result['mean_final_fidelity']:.6f} "
            f"(std={result['extras']['std_final_fidelity']:.6f})"
        )
    elif mode_name == 'adiabatic':
        fid_text = (
            f"final_fidelity={result['final_fidelity']:.6f} "
            f"path_fidelity={result['path_fidelity']:.6f}"
        )
    else:
        fid_text = f"final_fidelity={result['final_fidelity']:.6f}"
    print(
        f"  {mode_name:9s} | cost={result['cost']:.6e} | {fid_text} | runtime={result['runtime_s']:.2f}s"
    )


Final summary (costs and fidelities):
  terminal  | cost=2.494450e-02 | final_fidelity=0.999833 | runtime=8.55s
  ensemble  | cost=8.074746e-02 | final_fidelity=0.997871 mean_final_fidelity=0.990991 (std=0.011402) | runtime=254.32s
  adiabatic | cost=4.418770e-02 | final_fidelity=0.995800 path_fidelity=0.983920 | runtime=9.92s


## Changelog
- Terminal mode: configured linear CRAB + Adam for terminal fidelity, logging cost/fidelity/power/neg penalties, gradient norms, and per-iteration runtimes.
- Ensemble mode: implemented 51Ã—51 (Î², Doppler) grid averaging with chain-rule gradients and stored ensemble mean/std fidelity alongside runtime logs.
- Adiabatic mode: added eigen-path projector objective with custom adjoint recursion, tracking path fidelity and cost breakdown.
- Hyperparameters: Adam lr=0.08, Î²1=0.9, Î²2=0.999, eps=1e-8; stopping max_iters=450, target_cost=5e-5, max_oracle_calls=4e5.
- Deviations: no deviations from the guideâ€”penalties applied per spec and no smoothing filters introduced.
